**Задание 1**
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [42]:
import pandas as pd
df = pd.read_csv('ratings.csv')
df

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [67]:
users_100 = df.loc[df['userId'].value_counts()[df['userId'].value_counts()>100]]
users_100

,userId,movieId,rating,timestamp
2391,15,87222,4.0,1458506624
1868,15,4974,2.0,1349622574
1735,15,3988,1.0,1166587057
1700,15,3798,2.0,997937999
1610,15,3261,4.0,1058250521
...,...,...,...,...
104,3,377,2.5,1298923242
104,3,377,2.5,1298923242
103,3,356,5.0,1298862167
103,3,356,5.0,1298862167


In [87]:
result = users_100.groupby('userId').agg(['min', 'max'])['timestamp']
result

,min,max
userId,,
3,1298861605,1298932787
4,949778714,949982274
5,1163373293,1163375025
6,1108134311,1109258250
7,851866704,851869291
8,1154389528,1154473022
9,938628843,938629747
11,1391657561,1391657561
12,968045074,968045438


In [88]:
result.loc[:, 'cycle'] = result['max'] - result['min']
result

,min,max,cycle
userId,,,
3,1298861605,1298932787,71182
4,949778714,949982274,203560
5,1163373293,1163375025,1732
6,1108134311,1109258250,1123939
7,851866704,851869291,2587
8,1154389528,1154473022,83494
9,938628843,938629747,904
11,1391657561,1391657561,0
12,968045074,968045438,364


**Задание  2**
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [16]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [24]:
client_air = client_base.merge(air, how='left', on='client_id')
client_air_auto = client_air.merge(auto, how='left', on='client_id')
client_all = client_air_auto.merge(rzd, how='left', on='client_id')
client_all

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,NaN,57483.0,10283.0
3,114,Мира 14,NaN,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,NaN
6,117,Панфиловская 33,13.0,98.0,NaN
7,118,Мастеркова 4,173.0,NaN,NaN


In [25]:
del client_all['address']
client_all

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,NaN,NaN,1093.0
1,112,NaN,NaN,2810.0
2,113,NaN,57483.0,10283.0
3,114,NaN,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,NaN
6,117,13.0,98.0,NaN
7,118,173.0,NaN,NaN


**Задание 3**
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы.

Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Чтобы получить координаты пользователей при визите и покупке, нужно объединить датафреймы по столбцу timestamp. То есть к датафрейму с визитами и покупками присоединить (merge) датафрейм с координатами, используя параметры how='left' и on='timestamp'.

2. Не понимаю вопрос. Видимо, я неправильно поняла первый вопрос. Но о сквозной аналитике я совсем ничего не знаю.

3. Координаты показывают, где находятся пользователи, которые покупают нечто в приложении. Вероятно, можно преобразовать координаты в названия городов. А по списку городов понять, хорошо ли работает геотаргетинг, в каких (неучтенных) городах есть спрос и т.д.